In [1]:
import findspark
findspark.init()

In [2]:
spark_url = 'local'

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import ArrayType, StructType

In [4]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Clean Scraped Data')\
        .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/04 23:44:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/05/04 23:44:44 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [8]:
from kafka import KafkaProducer

In [28]:
kafka_broker = 'localhost:9093'

In [10]:
sc = spark.sparkContext

In [11]:
api_key = "11c74ac47ee301629d4f6f3a51e99a60"


In [12]:
import requests
import json

In [29]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [34]:
for round in range(100):
    url = f"https://api.elsevier.com/content/search/scopus?query=all(Chulalongkorn)&date=2017&view=COMPLETE&apiKey={api_key}&count=25&start={round * 25}"
    headers = {'content-type': 'applicaton/json'}

    res = requests.get(url, headers=headers)

    entries = res.json()["search-results"]["entry"]

    scopus_ids = [ entry["dc:identifier"] for entry in entries]
    for id in scopus_ids:
        url = f"https://api.elsevier.com/content/abstract/scopus_id/{id}?view=FULL&httpAccept=application%2Fjson&apiKey={api_key}&field=affiliation,author,citedby-count,subject-area,dc:title"
        res = requests.get(url)

        df = spark.read.option("multiline", True).json(sc.parallelize([res.text]))

        base_path = "abstracts-retrieval-response"
        try: 
            is_authors_struct = isinstance(df.schema[base_path].dataType["authors"].dataType["author"].dataType, StructType)
            is_affiliations_struct = isinstance(df.schema[base_path].dataType["affiliation"].dataType, StructType)

            df = (df.withColumn("title", df[base_path + ".coredata.dc:title"])
            .withColumn("id", F.lit(id))
            .withColumn("subject-area",F.col(base_path + ".subject-areas.subject-area.$"))
            .withColumn("year", F.lit("2017"))
            .withColumn("citation_count", F.col(base_path + ".coredata.citedby-count"))
            )

            if is_authors_struct:
                authors_array = F.array(df[base_path + ".authors.author"])
                df = df.withColumn("authors", authors_array)
            else:
                df = df.withColumn("authors", F.col(base_path + ".authors.author"))
            

            if is_affiliations_struct:
                affiliations_array = F.array(df[base_path + ".affiliation"])
                df = df.withColumn("affiliations", affiliations_array)
            else:
                df = df.withColumn("affiliations", F.col(base_path + ".affiliation"))
            


            df = df.drop(base_path)
            json_res = df.toJSON().first()
            producer.send('abstract', json_res.encode('utf-8'))

            df.write.mode("append").json("../rawData/webscrape")
        except Exception as e:
            print(f"Error processing: {e}")